In [1]:
'''

Code to add GNN encodings to T5 encoder
but GNN encoding require edge indices computation.
We tried creating them by using some cosine similarities over t5 encodings of input with schema
however this approach was not continued further
'''
import torch
import torch.nn as nn
from tokenizers import AddedToken
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import BertModel, T5ForConditionalGeneration, T5Tokenizer

C:\Users\Atharva\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Define the BERT-T5 model
class BertT5(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.t5 = T5ForConditionalGeneration.from_pretrained('t5-small')

    def forward(self, input_ids, attention_mask,
                        decoder_input_ids,
                        decoder_attention_mask,
                        labels=None):

        bert_outputs = self.bert(input_ids=input_ids,
                                 attention_mask=attention_mask)

        print((bert_outputs.last_hidden_state).size())

        t5_outputs = self.t5(decoder_input_ids=decoder_input_ids,
                             decoder_attention_mask=decoder_attention_mask,
                             encoder_outputs=bert_outputs.last_hidden_state,
#                              encoder_hidden_states=bert_outputs.last_hidden_state
                            )

        if labels is not None:
            loss = t5_outputs.loss(labels=labels)
            return loss
        else:
            return t5_outputs.logits

# Instantiate the BERT-T5 model and move it to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertT5().to(device)

# Load the data and tokenizer
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
text2sql_tokenizer = T5TokenizerFast.from_pretrained(
    't5-small',
    add_prefix_space = True
)
# TODO: Load and preprocess the data

# Define the optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

C:\Users\Atharva\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Atharva\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from load_dataset import Text2SQLDataset
from torch.utils.data import DataLoader

train_filepath = "../data/resdsql_pre/preprocessed_dataset_test.json"
batch_size = 2 #'input batch size.')

train_dataset = Text2SQLDataset(
        dir_ = train_filepath,
        mode = "train")

train_dataloder = DataLoader(
        train_dataset,
        batch_size = batch_size,
        shuffle = True,
        collate_fn = lambda x: x,
        drop_last = True
    )

if isinstance(text2sql_tokenizer, T5TokenizerFast):
    text2sql_tokenizer.add_tokens([AddedToken(" <="), AddedToken(" <")])

In [4]:
num_epochs = 1

# Train the model
model.train()
for epoch in range(num_epochs):
    for idx, batch in enumerate(train_dataloder):

        batch_inputs = [data[0] for data in batch]
        batch_sqls = [data[1] for data in batch]

        if epoch == 0 and idx == 0:
            for batch_id in range(len(batch_inputs)):
                print(f"batch_inputs - {batch_inputs[batch_id]}")
                print(f"batch_sqls - {batch_sqls[batch_id]}")
#                 print("----------------------")

        tokenized_inputs = text2sql_tokenizer(
            batch_inputs,
            padding = "max_length",
            return_tensors = "pt",
            max_length = 512, #512, max_encoder_len
            truncation = True
        )

        with text2sql_tokenizer.as_target_tokenizer():
            tokenized_outputs = text2sql_tokenizer(
                batch_sqls,
                padding = "max_length",
                return_tensors = 'pt',
                max_length = 256, #256, max_decoder_len
                truncation = True
            )

        encoder_input_ids = tokenized_inputs["input_ids"].to(device)
        encoder_input_attention_mask = tokenized_inputs["attention_mask"].to(device)

        decoder_input_ids = tokenized_outputs["input_ids"].to(device)
        decoder_attention_mask = tokenized_outputs["attention_mask"].to(device)
        labels = None #tokenized_outputs["attention_mask"].to(device)

        optimizer.zero_grad()
        loss = model(input_ids=encoder_input_ids,
                     attention_mask=encoder_input_attention_mask,
                     decoder_input_ids=decoder_input_ids,
                     decoder_attention_mask=decoder_attention_mask,
                     labels=labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        break

# Save the trained model
# torch.save(model.state_dict(), 'bert_t5_model.pt')

batch_inputs - What are the distinct names of wines that have appellations in the North Coast area?
batch_sqls - select distinct wine.name from appellations join wine on appellations.appelation = wine.appelation where appellations.area = 'North Coast'
batch_inputs - Find the number of distinct projects.
batch_sqls - select count ( distinct name ) from projects


C:\Users\Atharva\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


torch.Size([2, 512, 768])


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# Relation Graph Attention Network RGAT
